# Advanced Prompt Engineering Techniques

In this notebook, we'll explore various prompt engineering techniques that can significantly improve the performance of Large Language Models (LLMs). Prompt engineering is the art and science of designing effective prompts to elicit the best possible responses from LLMs.

We'll cover the following techniques:
1. Zero-shot Prompting
2. Few-shot Prompting
3. Chain of Thought (CoT) Prompting
4. Role Playing

Let's start by importing the necessary libraries.

In [ ]:
# Import required libraries
import os
import json
import requests
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
from google import genai

# Load environment variables from .env file (if you have any API keys)
load_dotenv()

# Configure the Google Generative AI API with your API key
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

## Helper Functions

Let's create some utility functions to make it easier to interact with the language model and display results consistently throughout the notebook.

In [ ]:
def get_gemini_response(prompt, model="gemini-2.0-flash"):
    """
    Get a response from Google's Gemini model
    
    Args:
        prompt (str): The prompt to send to the model
        model (str): The model to use (default: gemini-pro)
        
    Returns:
        str: The model's response
    """
    try:
        # Configure the model
        generation_config = {
            "temperature": 0.2,
            "top_p": 0.95,
            "top_k": 40,
            "max_output_tokens": 1024,
        }
        
        
        # Generate the response
        response = client.models.generate_content(
            contents=prompt,
            model=model,
            config=generation_config,
            )
        
        # Return the response text
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"

def display_prompt_response(prompt, response, technique=""):
    """
    Display the prompt and response in a formatted way
    
    Args:
        prompt (str): The prompt sent to the model
        response (str): The model's response
        technique (str): The prompt engineering technique used
    """
    print(f"{'='*80}")
    print(f"TECHNIQUE: {technique}")
    print(f"{'='*80}")
    print("\n📝 PROMPT:")
    print(f"{'-'*80}")
    print(prompt)
    print(f"{'-'*80}")
    print("\n🤖 RESPONSE:")
    print(f"{'-'*80}")
    print(response)
    print(f"{'-'*80}\n")

## 1. Zero-shot Prompting

Zero-shot prompting is a technique where we ask the model to perform a task without providing any examples. This approach relies on the model's pre-trained knowledge to understand and complete the task.

**Key characteristics:**
- No examples are provided
- The model relies entirely on its pre-trained knowledge
- Clear and specific instructions are crucial
- Works best for simple tasks or when the model has strong prior knowledge

Let's see some examples of zero-shot prompting:

In [ ]:
# Example 1: Classification task with zero-shot
zero_shot_classification = """
Classify the following text into one of these categories: Business, Technology, Sports, Politics, Entertainment.

Text: "Apple announces new iPhone with revolutionary AI capabilities and improved battery life."
Category:
"""

# Example 2: Translation task with zero-shot
zero_shot_translation = """
Translate the following English text to French:

"The artificial intelligence revolution is transforming how we work and live."
"""

# Example 3: Text summarization with zero-shot
zero_shot_summarization = """
Summarize the following paragraph in one sentence:

The James Webb Space Telescope, launched in December 2021, is the largest optical telescope in space. It conducts infrared astronomy and can view objects too old, distant, or faint for the Hubble Space Telescope. Its improved infrared resolution and sensitivity allow it to view objects too early in the universe's history for Hubble to see, which is expected to enable a broad range of investigations across many fields of astronomy and cosmology.
"""

# Let's test these zero-shot prompts
# Uncomment to run:
response1 = get_gemini_response(zero_shot_classification)
display_prompt_response(zero_shot_classification, response1, "Zero-shot Classification")

# response2 = get_gemini_response(zero_shot_translation)
# display_prompt_response(zero_shot_translation, response2, "Zero-shot Translation")

# response3 = get_gemini_response(zero_shot_summarization)
# display_prompt_response(zero_shot_summarization, response3, "Zero-shot Summarization")

## 2. Few-shot Prompting

Few-shot prompting involves providing the model with a few examples of the task before asking it to perform a similar task. This technique helps the model understand the pattern and expected output format.

**Key characteristics:**
- Includes a small number of examples (typically 2-5)
- Examples demonstrate the expected input-output pattern
- Helps the model understand the task's structure
- Improves performance on tasks where zero-shot might struggle
- Format consistency between examples and the final question is important

Let's see some examples of few-shot prompting:

In [ ]:
# Example 1: Sentiment analysis with few-shot learning
few_shot_sentiment = """
I'll provide some examples of sentiment analysis, then I want you to classify a new text.

Text: "The food was terrible and the service was even worse."
Sentiment: Negative

Text: "I had a great time at the concert, the band was amazing!"
Sentiment: Positive

Text: "The hotel room was clean, but the wifi didn't work properly."
Sentiment: Mixed

Text: "The new smartphone has impressive features but is quite expensive."
Sentiment:
"""

# Example 2: Named entity recognition with few-shot
few_shot_ner = """
Here are some examples of identifying organizations in text:

Text: "Apple is planning to open a new store in Mumbai next year."
Organization: Apple

Text: "Google and Microsoft announced a new partnership yesterday."
Organizations: Google, Microsoft

Text: "The World Health Organization issued new guidelines for pandemic preparedness."
Organization: World Health Organization

Text: "Tesla has revealed its latest electric vehicle model at a special event in Austin."
Organization:
"""

# Example 3: Custom classification with few-shot
few_shot_custom = """
Here are some examples of classifying customer feedback by department:

Feedback: "The website kept crashing when I tried to checkout."
Department: Technical Support

Feedback: "The representative was very helpful and solved my issue quickly."
Department: Customer Service

Feedback: "My package arrived damaged and some items were missing."
Department: Shipping

Feedback: "I was charged twice for my subscription."
Department:
"""

# Let's test these few-shot prompts
# Uncomment to run:
response4 = get_gemini_response(few_shot_sentiment)
display_prompt_response(few_shot_sentiment, response4, "Few-shot Sentiment Analysis")

# response5 = get_gemini_response(few_shot_ner)
# display_prompt_response(few_shot_ner, response5, "Few-shot Named Entity Recognition")

# response6 = get_gemini_response(few_shot_custom)
# display_prompt_response(few_shot_custom, response6, "Few-shot Custom Classification")

## 3. Chain of Thought (CoT) Prompting

Chain of Thought prompting is a technique that encourages the model to show its reasoning process step by step before arriving at the final answer. This approach often improves the accuracy of complex reasoning tasks.

**Key characteristics:**
- Explicitly asks the model to "think step by step"
- Breaks down complex problems into smaller, more manageable steps
- Can be combined with few-shot examples to demonstrate the reasoning process
- Particularly effective for mathematical problems, logical reasoning, and multi-step tasks
- Helps the model avoid jumping to incorrect conclusions

Let's see some examples of Chain of Thought prompting:

In [ ]:
# Example 1: Zero-shot Chain of Thought (adding "Let's think step by step")
zero_shot_cot = """
A store is having a 25% off sale. If a shirt originally costs $60 and jeans originally cost $90, how much would you save in total if you bought both items during the sale?

Let's think step by step.
"""

# Example 2: Few-shot Chain of Thought (providing example reasoning)
few_shot_cot = """
I'll solve some math problems by showing my reasoning step by step.

Problem: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?
Solution: Roger starts with 5 tennis balls. He buys 2 cans of tennis balls, with 3 tennis balls per can. So he buys 2 * 3 = 6 new tennis balls. Now he has 5 + 6 = 11 tennis balls.

Problem: A restaurant offers a 15% discount on the bill if you pay with cash. If your bill is $80, how much would you pay if you use cash?
Solution: The discount is 15% of $80. 15% of 80 is 80 * 0.15 = $12. So the discount is $12. The discounted price is $80 - $12 = $68. So you would pay $68.

Problem: Sarah is planning a trip to visit 3 different cities. She will spend 4 days in each city. She also spends 2 days traveling between each city. How many total days will Sarah's trip take?
Solution:
"""

# Example 3: Complex reasoning with CoT
complex_cot = """
Solve the following problem step by step: 

In a small town, 60% of adults work full-time, 15% work part-time, and the rest are unemployed or retired. If the town has 5,000 adults, and 20% of full-time workers and 30% of part-time workers also attend evening classes, how many adults in the town attend evening classes while also working (either full-time or part-time)?

Think through this step by step.
"""

# Let's test these Chain of Thought prompts
# Uncomment to run:
response7 = get_gemini_response(zero_shot_cot)
display_prompt_response(zero_shot_cot, response7, "Zero-shot Chain of Thought")

# response8 = get_gemini_response(few_shot_cot)
# display_prompt_response(few_shot_cot, response8, "Few-shot Chain of Thought")

# response9 = get_gemini_response(complex_cot)
# display_prompt_response(complex_cot, response9, "Complex Reasoning with Chain of Thought")

## 4. Role Playing Prompting

Role Playing prompting involves asking the model to adopt a specific persona or role when generating responses. By giving the model a specific role to play, you can guide its responses to align with the expertise, tone, and perspective of that role.

**Key characteristics:**
- Assigns a specific identity or role to the model
- Can include expertise level, personality traits, or professional background
- Often improves responses for specialized knowledge domains
- Helps maintain a consistent tone and perspective
- Can be combined with other techniques like Chain of Thought

Let's see some examples of Role Playing prompting:

In [ ]:
# Example 1: Expert role for technical explanation
expert_role = """
You are an experienced quantum physicist explaining quantum computing concepts to a computer science graduate student. 

Explain quantum superposition and entanglement, and how they enable quantum computing to solve certain problems more efficiently than classical computing.
"""

# Example 2: Historical figure role
historical_role = """
You are Ada Lovelace, the world's first computer programmer who worked with Charles Babbage on the Analytical Engine in the 1800s.

What would you think about modern artificial intelligence and machine learning? Respond in the style and with the perspective that Ada Lovelace might have had.
"""

# Example 3: Professional role with specific task
professional_role = """
You are a senior marketing strategist for a sustainable fashion brand targeting environmentally conscious consumers aged 25-40.

Develop a brief marketing strategy for launching a new line of clothing made from recycled ocean plastic. Include key messaging points, potential channels for promotion, and how to address potential consumer concerns.
"""

# Let's test these Role Playing prompts
# Uncomment to run:
# response10 = get_gemini_response(expert_role)
# display_prompt_response(expert_role, response10, "Expert Role - Quantum Physicist")

# response11 = get_gemini_response(historical_role)
# display_prompt_response(historical_role, response11, "Historical Role - Ada Lovelace")

# response12 = get_gemini_response(professional_role)
# display_prompt_response(professional_role, response12, "Professional Role - Marketing Strategist")

## Comparing Prompt Engineering Techniques

Let's create a comparison of the various prompt engineering techniques we've explored:

| Technique | Best Used For | Advantages | Limitations |
|-----------|---------------|------------|-------------|
| Zero-shot | Simple tasks, standard operations | No examples needed, minimal prompt | May struggle with complex or ambiguous tasks |
| Few-shot | Tasks with specific patterns or formats | Improves accuracy through example learning | Requires careful selection of representative examples |
| Chain of Thought | Complex reasoning, multi-step problems | Improves accuracy for logical/mathematical tasks | Creates longer outputs, may introduce verbosity |
| Role Playing | Domain-specific knowledge, consistent tone | Specialized knowledge, consistent perspective | May limit general knowledge if role is too narrow |

## Combining Techniques

These techniques are not mutually exclusive and can be combined for even better results. For example:

1. **Few-shot Chain of Thought**: Provide examples that demonstrate step-by-step reasoning, then ask the model to solve a new problem using the same approach.
2. **Role Playing with Chain of Thought**: Ask the model to adopt a specific role and then think through a problem step by step from that perspective.
3. **Zero-shot with Role Playing**: Give the model a role but no examples, relying on its pre-trained knowledge within that role's perspective.

## Conclusion

Prompt engineering is both an art and a science. The effectiveness of different techniques depends on:

- The specific task you're trying to accomplish
- The complexity of the problem
- The capabilities of the underlying model
- The clarity and structure of your prompt

Experimenting with different approaches and combinations of techniques will help you find the most effective way to communicate with language models for your specific use cases.

In [ ]:
# Practical Exercise: Compare techniques on the same task

# Let's define a task that we'll approach with different prompt engineering techniques
task_description = "Explain the concept of machine learning bias to a non-technical audience."

# Zero-shot approach
zero_shot_prompt = f"""
{task_description}
"""

# Few-shot approach
few_shot_prompt = f"""
Here are some examples of explanations for technical concepts:

Technical Concept: Cloud Computing
Explanation for Non-Technical Audience: Cloud computing is like renting space in someone else's digital garage. Instead of storing all your files and running programs on your own computer, you keep them on big computers owned by companies like Amazon or Microsoft. You access these files and programs through the internet. This saves you from buying and maintaining expensive equipment, and you can usually access your stuff from any device with internet access.

Technical Concept: Encryption
Explanation for Non-Technical Audience: Encryption is like sending a letter in a locked box. Only the person with the right key can open it and read the message inside. Even if someone intercepts the box during delivery, they can't read your message without the key. This is how your sensitive information, like credit card numbers, stays secure when you shop online.

{task_description}
"""

# Chain of thought approach
cot_prompt = f"""
{task_description}

Let's break this down step by step:
"""

# Role playing approach
role_prompt = f"""
You are a friendly science teacher who specializes in making complex technical concepts accessible to middle school students using simple analogies and everyday examples.

{task_description}
"""

# Combining approaches (Role playing + Chain of thought)
combined_prompt = f"""
You are a friendly science teacher who specializes in making complex technical concepts accessible to middle school students using simple analogies and everyday examples.

{task_description}

Let's break this down step by step with some relatable examples:
"""

# Uncomment these to run the comparisons
# comparisons = [
#     ("Zero-shot", zero_shot_prompt),
#     ("Few-shot", few_shot_prompt),
#     ("Chain of Thought", cot_prompt),
#     ("Role Playing", role_prompt),
#     ("Combined Approach", combined_prompt)
# ]

# for name, prompt in comparisons:
#     response = get_gemini_response(prompt)
#     display_prompt_response(prompt, response, f"{name} Approach")

## Additional Resources and References

Here are some valuable resources for further exploration of prompt engineering techniques:

### Research Papers
- Wei, J., Wang, X., Schuurmans, D., et al. (2022). "[Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/abs/2201.11903)"
- Brown, T. B., Mann, B., Ryder, N., et al. (2020). "[Language Models are Few-Shot Learners](https://arxiv.org/abs/2005.14165)"
- Kojima, T., Gu, S. S., Reid, M., et al. (2022). "[Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)"

### Guides and Tutorials
- [Google's Gemini API Documentation](https://ai.google.dev/gemini-api/docs)
- [OpenAI's GPT Best Practices Guide](https://platform.openai.com/docs/guides/prompt-engineering)
- [Anthropic's Guide to Prompt Design with Claude](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/overview#how-to-prompt-engineer)

### Community Resources
- [Prompt Engineering Guide](https://www.promptingguide.ai/)
- [Anthropic's Prompt Engineering Interactive Tutorial](https://github.com/anthropics/prompt-eng-interactive-tutorial)

Remember that prompt engineering is an evolving field, and new techniques and best practices continue to emerge as language models and their applications develop.